In [48]:
#Position-value dependent

#Unpack necessities
from datetime import datetime
import pandas as pd
import numpy as np
import my_config
import schedule
import time
import ccxt

import warnings
warnings.filterwarnings('ignore')

#Elimanates time/order issues, not really sure why this works...
ccxt.binanceus({ 'options':{ 'adjustForTimeDifference':True}})

exchange = ccxt.binanceus({
"apiKey": my_config.BINANCE_KEY,
"secret": my_config.BINANCE_SECRET,
'enableRateLimit': True})

#Super trend formula.
def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=7, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

####Parameters for HARD-VALUE TRADE AMOUNT####
#in_position = False
#ticker = 'HBAR/USD'
#trade_amount = 1000
#bar = exchange.fetch_ohlcv(f'{ticker}', timeframe='1m', limit=5)
#order_size = int(trade_amount/bar[4][1]-(.05*(trade_amount/bar[4][1]))) #Trades $1000.

#Parameters for POSITION-DEPENDENT TRADE AMOUNT
ticker = input("Insert ticker (XXX/USD): ")+'/USD'

#Get position_val
bal = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
bal['free'] = pd.to_numeric(bal['free'])
bal = bal[bal.free!=0].drop(columns='locked').reset_index(drop=True)
bal = bal[bal['asset']==ticker[:4].replace('/','')].reset_index(drop=True).free[0]
in_position = int(bal)>0

#Percentage of dedicated trading value
perc = int(input("Percentage of tradeable amount: "))/100
order_size = int(bal*perc)

#Decision maker.
def check_buy_sell_signals(df):
    global in_position,order_size,ticker
    print("Analyzing",ticker,"data... \nIn_position:",in_position,'\n')
    print(df.tail(3)[['timestamp','open','in_uptrend']])
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1
    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
        print("Changed to uptrend - Buy")
        if not in_position:
            order = exchange.create_market_buy_order(f'{ticker}',order_size)
            print('Status:'+order['info']['status'],
                  'Price:'+order['trades'][0]['info']['price'],
                  'Quantity:'+order['info']['executedQty'],
                  'Type:'+order['info']['side'])
            in_position = True
        else:
            print("Already in position, no task.")
    
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        if in_position: 
            print("Changed to downtrend - Sell")
            order = exchange.create_market_sell_order(f'{ticker}',order_size)
            print('Status:'+order['info']['status'],
                  'Price:'+order['trades'][0]['info']['price'],
                  'Quantity:'+order['info']['executedQty'],
                  'Type:'+order['info']['side'])
            in_position = False
        else:
            print("No selling position, no task.")

In [49]:
#Run
def run_bot():
    print(f"\n\nFetching new bars for {datetime.now().isoformat()}")
    bars = exchange.fetch_ohlcv(f'{ticker}', timeframe='1m', limit=100)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    supertrend_data = supertrend(df)
    check_buy_sell_signals(supertrend_data)
    print()
schedule.every(3).seconds.do(run_bot)
while True:
    schedule.run_pending()
    time.sleep(1)
    

Fetching new bars for 2021-09-13T18:31:03.077266
Analyzing VTHO/USD data... 
In_position: True 

             timestamp    open  in_uptrend
21 2021-09-14 01:27:00  0.0096       False
22 2021-09-14 01:28:00  0.0097        True
23 2021-09-14 01:29:00  0.0097        True

Fetching new bars for 2021-09-13T18:31:03.205704
Analyzing VTHO/USD data... 
In_position: True 

             timestamp    open  in_uptrend
21 2021-09-14 01:27:00  0.0096       False
22 2021-09-14 01:28:00  0.0097        True
23 2021-09-14 01:29:00  0.0097        True



Fetching new bars for 2021-09-13T18:31:06.350414
Analyzing VTHO/USD data... 
In_position: True 

             timestamp    open  in_uptrend
96 2021-09-14 01:27:00  0.0096       False
97 2021-09-14 01:28:00  0.0097        True
98 2021-09-14 01:29:00  0.0097        True



Fetching new bars for 2021-09-13T18:31:09.510290
Analyzing VTHO/USD data... 
In_position: True 

             timestamp    open  in_uptrend
96 2021-09-14 01:28:00  0.0097        True
97 



Fetching new bars for 2021-09-13T18:32:17.369828
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:29:00  0.0097        True
97 2021-09-14 01:30:00  0.0097        True
98 2021-09-14 01:31:00  0.0097       False
No selling position, no task.



Fetching new bars for 2021-09-13T18:32:20.519207
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:29:00  0.0097        True
97 2021-09-14 01:30:00  0.0097        True
98 2021-09-14 01:31:00  0.0097       False
No selling position, no task.


Fetching new bars for 2021-09-13T18:32:21.679108
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
21 2021-09-14 01:29:00  0.0097        True
22 2021-09-14 01:30:00  0.0097        True
23 2021-09-14 01:31:00  0.0097       False
No selling position, no task.



Fetching new bars for 2021-09-13T18:32:23.839055
Analyzing VTHO/USD data... 
In_position: 

Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:30:00  0.0097        True
97 2021-09-14 01:31:00  0.0097       False
98 2021-09-14 01:32:00  0.0096       False



Fetching new bars for 2021-09-13T18:33:28.638646
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:30:00  0.0097        True
97 2021-09-14 01:31:00  0.0097       False
98 2021-09-14 01:32:00  0.0096       False



Fetching new bars for 2021-09-13T18:33:31.790777
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:30:00  0.0097        True
97 2021-09-14 01:31:00  0.0097       False
98 2021-09-14 01:32:00  0.0096       False



Fetching new bars for 2021-09-13T18:33:34.950541
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:30:00  0.0097        True
97 2021-09-14 01:31:00  0.0097       False
98 



Fetching new bars for 2021-09-13T18:34:42.531138
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:31:00  0.0097       False
97 2021-09-14 01:32:00  0.0096       False
98 2021-09-14 01:33:00  0.0097       False


Fetching new bars for 2021-09-13T18:34:44.719098
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
21 2021-09-14 01:31:00  0.0097       False
22 2021-09-14 01:32:00  0.0096       False
23 2021-09-14 01:33:00  0.0097       False



Fetching new bars for 2021-09-13T18:34:45.857252
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:31:00  0.0097       False
97 2021-09-14 01:32:00  0.0096       False
98 2021-09-14 01:33:00  0.0097       False



Fetching new bars for 2021-09-13T18:34:49.036151
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:31:00  0.0097       Fa



Fetching new bars for 2021-09-13T18:36:00.063053
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:32:00  0.0096       False
97 2021-09-14 01:33:00  0.0097       False
98 2021-09-14 01:34:00  0.0097       False



Fetching new bars for 2021-09-13T18:36:03.299592
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:32:00  0.0096       False
97 2021-09-14 01:33:00  0.0097       False
98 2021-09-14 01:34:00  0.0097       False


Fetching new bars for 2021-09-13T18:36:04.494957
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
21 2021-09-14 01:32:00  0.0096       False
22 2021-09-14 01:33:00  0.0097       False
23 2021-09-14 01:34:00  0.0097       False



Fetching new bars for 2021-09-13T18:36:06.646547
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:32:00  0.0096       Fa

...

Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:41:00  0.0097       False
97 2021-09-14 01:42:00  0.0097       False
98 2021-09-14 01:43:00  0.0096       False



Fetching new bars for 2021-09-13T18:44:57.675337
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:41:00  0.0097       False
97 2021-09-14 01:42:00  0.0097       False
98 2021-09-14 01:43:00  0.0096       False



Fetching new bars for 2021-09-13T18:45:00.900408
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:41:00  0.0097       False
97 2021-09-14 01:42:00  0.0097       False
98 2021-09-14 01:43:00  0.0096       False


Fetching new bars for 2021-09-13T18:45:04.069598
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
21 2021-09-14 01:41:00  0.0097       False
22 2021-09-14 01:42:00  0.0097       False
23 2



Fetching new bars for 2021-09-13T18:46:12.042046
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:43:00  0.0096       False
97 2021-09-14 01:44:00  0.0096       False
98 2021-09-14 01:45:00  0.0096       False



Fetching new bars for 2021-09-13T18:46:15.216003
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:43:00  0.0096       False
97 2021-09-14 01:44:00  0.0096       False
98 2021-09-14 01:45:00  0.0096       False



Fetching new bars for 2021-09-13T18:46:18.421104
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:43:00  0.0096       False
97 2021-09-14 01:44:00  0.0096       False
98 2021-09-14 01:45:00  0.0096       False



Fetching new bars for 2021-09-13T18:46:21.605220
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:43:00  0.0096       F

Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
21 2021-09-14 01:44:00  0.0096       False
22 2021-09-14 01:45:00  0.0096       False
23 2021-09-14 01:46:00  0.0096       False



Fetching new bars for 2021-09-13T18:47:29.658628
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:44:00  0.0096       False
97 2021-09-14 01:45:00  0.0096       False
98 2021-09-14 01:46:00  0.0096       False



Fetching new bars for 2021-09-13T18:47:32.827071
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:44:00  0.0096       False
97 2021-09-14 01:45:00  0.0096       False
98 2021-09-14 01:46:00  0.0096       False



Fetching new bars for 2021-09-13T18:47:36.003864
Analyzing VTHO/USD data... 
In_position: False 

             timestamp    open  in_uptrend
96 2021-09-14 01:44:00  0.0096       False
97 2021-09-14 01:45:00  0.0096       False
98 



Fetching new bars for 2021-09-13T18:48:37.492519
Analyzing VTHO/USD data... 
In_position: True 

             timestamp    open  in_uptrend
96 2021-09-14 01:45:00  0.0096       False
97 2021-09-14 01:46:00  0.0096       False
98 2021-09-14 01:47:00  0.0097        True
Changed to uptrend - Buy
Already in position, no task.



Fetching new bars for 2021-09-13T18:48:40.669858
Analyzing VTHO/USD data... 
In_position: True 

             timestamp    open  in_uptrend
96 2021-09-14 01:45:00  0.0096       False
97 2021-09-14 01:46:00  0.0096       False
98 2021-09-14 01:47:00  0.0097        True
Changed to uptrend - Buy
Already in position, no task.



Fetching new bars for 2021-09-13T18:48:43.873379
Analyzing VTHO/USD data... 
In_position: True 

             timestamp    open  in_uptrend
96 2021-09-14 01:45:00  0.0096       False
97 2021-09-14 01:46:00  0.0096       False
98 2021-09-14 01:47:00  0.0097        True
Changed to uptrend - Buy
Already in position, no task.



Fetching new bars

KeyboardInterrupt: 

In [ ]:
#Keep running...

#Needs: 
#visualization
#balance tracker

In [46]:
#Get position_val
bal = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
bal['free'] = pd.to_numeric(bal['free'])
bal = bal[bal.free!=0].drop(columns='locked').reset_index(drop=True)
bal = bal[bal['asset']==ticker[:4]].reset_index(drop=True).free[0]
in_position = bal>0
in_position

True

In [151]:
bars = exchange.fetch_ohlcv('VTHO/USD', timeframe='1m', limit=5)
int(10/bars[4][1]-(.05*(10/bars[4][1])))

1000

In [120]:
exchange.create_market_buy_order('VTHO/USD',1100)

{'info': {'symbol': 'VTHOUSD',
  'orderId': '21356239',
  'orderListId': '-1',
  'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
  'transactTime': '1631518396580',
  'price': '0.0000',
  'origQty': '1100.00000000',
  'executedQty': '1100.00000000',
  'cummulativeQuoteQty': '10.4500',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'fills': [{'price': '0.0095',
    'qty': '1100.00000000',
    'commission': '0.00001992',
    'commissionAsset': 'BNB',
    'tradeId': '4227218'}]},
 'id': '21356239',
 'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
 'timestamp': 1631518396580,
 'datetime': '2021-09-13T07:33:16.580Z',
 'lastTradeTimestamp': None,
 'symbol': 'VTHO/USD',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 0.0095,
 'stopPrice': None,
 'amount': 1100.0,
 'cost': 10.45,
 'average': 0.0095,
 'filled': 1100.0,
 'remaining': 0.0,
 'status': 'closed',
 'fee': {'cost': 1.992e-05, 'currency': 'BNB'},
 't

In [121]:
order = {'info': {'symbol': 'VTHOUSD',
  'orderId': '21356239',
  'orderListId': '-1',
  'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
  'transactTime': '1631518396580',
  'price': '0.0000',
  'origQty': '1100.00000000',
  'executedQty': '1100.00000000',
  'cummulativeQuoteQty': '10.4500',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'fills': [{'price': '0.0095',
    'qty': '1100.00000000',
    'commission': '0.00001992',
    'commissionAsset': 'BNB',
    'tradeId': '4227218'}]},
 'id': '21356239',
 'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
 'timestamp': 1631518396580,
 'datetime': '2021-09-13T07:33:16.580Z',
 'lastTradeTimestamp': None,
 'symbol': 'VTHO/USD',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 0.0095,
 'stopPrice': None,
 'amount': 1100.0,
 'cost': 10.45,
 'average': 0.0095,
 'filled': 1100.0,
 'remaining': 0.0,
 'status': 'closed',
 'fee': {'cost': 1.992e-05, 'currency': 'BNB'},
 'trades': [{'info': {'price': '0.0095',
    'qty': '1100.00000000',
    'commission': '0.00001992',
    'commissionAsset': 'BNB',
    'tradeId': '4227218'},
   'timestamp': None,
   'datetime': None,
   'symbol': 'VTHO/USD',
   'id': None,
   'order': None,
   'type': None,
   'side': None,
   'takerOrMaker': None,
   'price': 0.0095,
   'amount': 1100.0,
   'cost': 10.45,
   'fee': {'cost': 1.992e-05, 'currency': 'BNB'}}],
 'fees': [{'cost': 1.992e-05, 'currency': 'BNB'}]}

In [146]:
#Order details. printable.
order['info']['symbol'],order['info']['orderId'], order['info']['status'], order['trades'][0]['info']['price'], order['info']['executedQty']

('VTHOUSD', '21356239', 'FILLED', '0.0095', '1100.00000000')

In [145]:
order['trades'][0]['info']['price']

'0.0095'

# Set condition(s) for variable: in_position

In [18]:
df = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
df['free'] = pd.to_numeric(df['free'])
df['locked'] = pd.to_numeric(df['locked'])
df = df[df.free!=0].drop(columns='locked').reset_index(drop=True)

In [31]:
df[df['asset']=='HBAR'].free

8    3124.07
Name: free, dtype: float64

In [67]:
#Parameters for POSITION-DEPENDENT TRADE AMOUNT
ticker = input("Insert ticker (XXX/USD): ")+'/USD'

#Get position_val
bal = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
bal['free'] = pd.to_numeric(bal['free'])
bal = bal[bal.free!=0].drop(columns='locked').reset_index(drop=True)
bal = bal[bal['asset']==ticker[:4].replace('/','')].reset_index(drop=True).free[0]
in_position = int(bal)>0
in_position

Insert ticker (XXX/USD): RVN


True

In [65]:
bal[bal['asset']==ticker[:4].replace('/','')].reset_index(drop=True).free[0]

numpy.float64

In [54]:
ticker[:4]

'RVN/'